# Data Exploration with Python and Jupyter - part 3

Basic usage of the Pandas library to download a dataset,
explore its contents, clean up missing or invalid data,
filter the data according to different criteria,
and plot visualizations of the data.

- [Part 1: Python and Jupyter](https://ssciwr.github.io/jupyter-data-exploration)
- [Part 2: Pandas with toy data](https://ssciwr.github.io/jupyter-data-exploration/pandas-toy-data.html)
- **Part 3: Pandas with real data**

*Press `Spacebar` to go to the next slide (or `?` to see all navigation shortcuts)*

# Let's download some real data

For some reason, the London Fire Brigade provides a public spreadsheet of all animal rescue incidents since 2009:

https://data.london.gov.uk/dataset/animal-rescue-incidents-attended-by-lfb

They provide a link to the dataset in excel format

In [ ]:
# import the Pandas library & matplotlib for plotting

import pandas as pd
import matplotlib.pyplot as plt

# also import geopandas for later use
import geopandas

In [ ]:
# download an excel spreadsheet with some data and convert it to a DataFrame
url = "https://data.london.gov.uk/download/animal-rescue-incidents-attended-by-lfb/01007433-55c2-4b8a-b799-626d9e3bc284/Animal%20Rescue%20incidents%20attended%20by%20LFB%20from%20Jan%202009.csv.xlsx"
df = pd.read_excel(url)

## Suggested workflow / philosophy

- you want to do something
  - if you know / have a guess which function to use, look at its docstring: `?function_name`
  - if you don't have any idea what to try, google `how do I ... in pandas`
  - if in doubt, just try something!
- if you get an error, copy & paste the last bit into google (along with `funtion_name` and/or `pandas`)
  - don't be intimidated by the long and apparently nonsensical error messages
  - almost certainly someone else has had this exact problem
  - almost certainly the solution is waiting for you
- look for a stackoverflow answer with many up-votes
  - ignore the green tick, this just means the person asking the question liked the answer
  - typically an answer with many up-votes is a better option
  - more recent answers can also be better: sometimes a library has changed since an older answer was written

(For anyone who wasn't already doing this, that may be the most useful thing in this course)

# Display the DataFrame

In [ ]:
df

# Column data types

In [ ]:
df.dtypes

# Convert DateTimeOfCall to a date-time

In [ ]:
df["DateTimeOfCall"].head()

In [ ]:
# this looks like what we want..
pd.to_datetime(df["DateTimeOfCall"]).head()

In [ ]:
# ..but which number is the month and which is the day?
# how can we check if what we just did was correct?
pd.to_datetime(df["DateTimeOfCall"]).plot()
# should be a single monotonically increasing line: looks good!

In [ ]:
# replace DateTimeOfCall column in dataframe with this one
df["DateTimeOfCall"] = pd.to_datetime(df["DateTimeOfCall"])

# Use the datetime as the index

In [ ]:
df.set_index("DateTimeOfCall", inplace=True)

In [ ]:
df

In [ ]:
# can now use datetime to select rows: here is jan 2021
df.loc["2021-01-01":"2021-01-31", "FinalDescription"]

In [ ]:
# resample the timeseries by month and count incidents
df.resample("ME")["IncidentNumber"].count().plot(title="Monthly Calls")
# see https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases
plt.show()

In [ ]:
# resample by year, sum total costs, average hourly costs
fig, axs = plt.subplots(figsize=(16, 4), ncols=2)
df.resample("YE")["IncidentNotionalCost(£)"].sum().plot(
    title="Year total cost", ax=axs[0]
)
df.resample("YE")["HourlyNotionalCost(£)"].mean().plot(
    title="Average hourly cost", ax=axs[1]
)
plt.show()

# Missing data

Different strategies for dealing with missing data:

- Ignore the issue
  - some things may break / not work as expected
- Remove rows/columns with missing data
  - remove all rows with missing data: `df.dropna(axis=0)`
  - remove all columns with missing data: `df.dropna(axis=1)`
- Guess (impute) missing data
  - replace all missing entries with a value: `df.fillna(1)`
  - replace missing entries with mean for that column `df.fillna(df.mean())`
  - replace each missing entry with previous valid entry: `df.fillna(method="pad")`
  - replace missing by interpolating between valid entries: `df.interpolate()`

In [ ]:
# count missing entries for each column
df.isna().sum()

In [ ]:
# If PumpCount is missing, typically so is PumpHoursTotal
# 66 rows are missing at least one of these
pump_missing = df["PumpCount"].isna() | df["PumpHoursTotal"].isna()
print(pump_missing.sum())

In [ ]:
# so we could choose to drop these rows
df1 = df.drop(df.loc[pump_missing].index)
# here we made a new dataset df1 with these rows dropped
# to drop the rows from the original dataset df, could do:
#
# df = df.drop(df.loc[pump_missing == True].index)
#
# or:
#
# df.drop(df.loc[pump_missing == True].index, inplace=True)
#
print(len(df1))

In [ ]:
# another equivalent way to do this
df2 = df.dropna(subset=["PumpCount", "PumpHoursTotal"])
print(len(df2))

In [ ]:
# but if we drop them, we lose valid data from other columns
# let's look at the distribution of values:
fig, axs = plt.subplots(1, 2, figsize=(14, 6))
df.plot.hist(y="PumpCount", ax=axs[0])
df.plot.hist(y="PumpHoursTotal", ax=axs[1])
plt.plot()

In [ ]:
# looks like it would be better to replace missing PumpCount and PumpHoursTotal fields with 1
?df.fillna
df.fillna({"PumpCount": 1, "PumpHoursTotal": 1}, inplace=True)

In [ ]:
df.isna().sum()

# Count the unique entries in each column

In [ ]:
df.nunique().sort_values()

In [ ]:
# "cat" and "Cat" are treated as different animals here:
df["AnimalGroupParent"].unique()

In [ ]:
# select rows where AnimalGroupParent is "cat", replace with "Cat"
df.loc[df["AnimalGroupParent"] == "cat", "AnimalGroupParent"] = "Cat"

In [ ]:
df["AnimalGroupParent"].unique()

In [ ]:
df.groupby("AnimalGroupParent")["IncidentNumber"].count().sort_values().plot.barh(
    logx=True
)
plt.show()

In [ ]:
# apparently different hourly costs
# does it depend on the type of event? or does it just increase over time?
df["HourlyNotionalCost(£)"].unique()

In [ ]:
# just goes up over time
df["HourlyNotionalCost(£)"].plot.line()

In [ ]:
# Group incidents by fire station & count them
df.groupby("StnGroundName")["IncidentNumber"].count()

## Plot location of calls on a map

- note: this section uses some more libraries, to install them:
- `pip install geopandas contextily`

In [ ]:
# drop missing longitude/latitude
df2 = df.dropna(subset=["Longitude", "Latitude"])
# also drop zero values
df2 = df2[df2["Latitude"] != 0]

# set crs to EPSG:4326 to specify WGS84 Latitude/Longitude
gdf = geopandas.GeoDataFrame(
    df2,
    geometry=geopandas.points_from_xy(df2["Longitude"], df2["Latitude"]),
    crs="EPSG:4326",
)
gdf.head()

In [ ]:
f, ax = plt.subplots(figsize=(16, 16))
# plot location of calls involving animals
gdf.plot(ax=ax, color="black", alpha=0.3)
plt.title("Call locations")
# plt.axis("off")
plt.show()

In [ ]:
import contextily as cx

f, ax = plt.subplots(figsize=(16, 16))
# plot location of calls involving animals
gdf.plot(ax=ax, color="black", alpha=0.3)
# add a basemap of the region using contextily
cx.add_basemap(ax, crs=gdf.crs)
plt.title("Call locations")
plt.axis("off")
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(16, 16))
# plot location of calls involving animals
for animal, colour in [
    ("Cow", "black"),
    ("Deer", "red"),
    ("Fox", "blue"),
    ("Snake", "yellow"),
]:
    gdf[gdf["AnimalGroupParent"] == animal].plot(
        ax=ax, color=colour, alpha=0.5, label=animal
    )
# add a basemap of the region using contextily
cx.add_basemap(ax, crs=gdf.crs)
plt.title("Call locations by animal")
plt.legend()
plt.axis("off")
plt.show()

# Next steps

- experiment with your own datasets
- read some pandas documentation
  - [user guide](https://pandas.pydata.org/docs/user_guide/index.html)
- follow a tutorial
  - [getting started tutorials](https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html)
- free interactive kaggle courses
  - [pandas](https://www.kaggle.com/learn/pandas)
  - [data cleaning](https://www.kaggle.com/learn/data-cleaning)